In [3]:
import numpy as np
import pandas as pd
import cv2
import pickle
import mediapipe as mp
from sklearn.preprocessing import StandardScaler


In [4]:
train_df = pd.read_csv('landmark_data/landmarks_train_v3.csv')
X_train = train_df.iloc[:, 1:].values
scaler = StandardScaler().fit(X_train)
with open('trained_models/random_forest_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [36]:
scaler = StandardScaler()

In [53]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

confidence = 0.5
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=confidence)

I0000 00:00:1709187065.743623   35630 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1709187065.744678   37330 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.0.4-0ubuntu1~22.04.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)


In [ ]:
cap = cv2.VideoCapture(0)


while cap.isOpened():
    ret, image = cap.read()

    if not ret:
        continue
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    results = hands.process(image)

    image.flags.writeable = True
    
    #converting the BGR image to RGB.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    both_hand_landmarks = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = []
            for landmark in hand_landmarks.landmark:
                # Extract x, y coordinates (relative to image dimensions)
                x = landmark.x
                y = landmark.y
                # Append coordinates to the list
                landmarks.append((x, y))
            both_hand_landmarks.append(landmarks)
        
        if len(both_hand_landmarks) == 1:
            both_hand_landmarks.append([(0, 0)] * len(both_hand_landmarks[0]))
        values = list(np.array(both_hand_landmarks).flatten())
        values = scaler.transform([values])
        predicted = loaded_model.predict(values)
        cv2.rectangle(image, (0,0), (160, 60), (245, 90, 16), -1)
        # Displaying Class
        cv2.putText(image, 'Predicted Gesture'
                    , (20,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(predicted[0])
                    , (20,45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.imshow('Sign Translator', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [55]:
cap.release()
cv2.destroyAllWindows()